In [1]:
QUERY = "How can I write an article on WEF’s blog?"

In [2]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, Markdown

from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [3]:
# Make sure to update the allowed IP addresses in the Google Custom Search Engine
load_dotenv()
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Point to the local server
# client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

client = OpenAI(base_url="https://api.openai.com/v1", api_key=os.getenv("OPEN_AI_KEY"))

In [4]:
search = GoogleSearchAPIWrapper()

def top5_results(query):
    return search.results(query, 5)

tool = Tool(
    name="Google Search Snippets",
    description="Search Google for recent results.",
    func=top5_results,
)

In [5]:
google_search_results = tool.run(QUERY)

In [6]:
def fetch_page_text_with_uc(url):
    try:
        # Initialize the Chrome driver
        options = uc.ChromeOptions()
        
        # If you want Chrome to run headless (without opening a window)
        options.add_argument('--headless')
        
        driver = uc.Chrome(options=options)

        # Navigate to the page
        driver.get(url)
        
        # Wait for the page to load and ensure a certain element is present
        # Adjust the below line to wait for a specific element that indicates the page has loaded
        # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        
        # Now that the page is loaded, you can extract its text
        page_text = driver.find_element(By.TAG_NAME, "body").text
        
        return page_text
        
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Make sure to quit the driver to free up resources
        driver.quit()

In [7]:
completion = client.chat.completions.create(
  model="gpt-4-0125-preview",
  messages=[
    {"role": "system", "content": "As the WEF_and_Global_Shapers_Expert, my role is to provide precise information on the World Economic Forum (WEF) and the Global Shapers Community. I synthesize data on WEF's missions and the impactful projects of Global Shapers, ensuring users receive the latest and most relevant insights. To achieve this, I will initially use text from official websites for responses. If the available text does not sufficiently answer a question, I will rely on my own knowledge, ensuring accuracy by only providing information I am 100% certain of. In cases where I cannot confirm the accuracy of my knowledge or the relevance of website content, I will honestly report having no information on the topic."},
    {"role": "user", "content": QUERY},
    {"role": "system", "content": "Website Body:" + fetch_page_text_with_uc(google_search_results[0]["link"])},
  ],
  temperature=0.7,
)

display(Markdown(completion.choices[0].message.content))

To write an article for the World Economic Forum's (WEF) Agenda blog, you should follow these steps:

1. **Develop a Unique and Engaging Idea:** Your article should cover a topic that hasn't been extensively featured on Agenda before. It should be engaging and written in a way that appeals to a broad audience, even if it deals with a niche subject. Your piece should aim to make complex global issues accessible to everyone.

2. **Prepare Your Pitch:** Your pitch should be detailed and outline the three key points you intend to address in your article. It's crucial to familiarize yourself with recent Agenda articles related to your topic to avoid pitching duplicate ideas. Explore various successful formats like provoking Op-Eds, numbered lists, predictions, explainer-style pieces, summaries of research, or questions to be answered.

3. **Submit Your Pitch:** Use the provided portal to submit your pitch. Ensure that your pitch is well thought out and clearly outlines what you plan to discuss in your article.

4. **Wait for a Response:** The WEF editorial team aims to respond to pitches within two weeks for articles not tied to a WEF event or meeting. If your article is tied to a WEF event, make sure to adhere to the specific deadlines for submitting pitches and final drafts.

5. **Article Submission:** If your pitch is accepted, you'll be invited to submit your final draft via email. The article should be between 800-1,000 words, well-written, and well-argued. It should not presume specialist knowledge and must be accessible to an intelligent reader. Include hyperlinks to your source material and at least one chart, figure, or graphic. Use specific examples or personal experiences to back up your argument.

6. **Publishing Terms:** Be aware of the WEF's publishing terms. They publish under Creative Commons and share the right to republish with non-commercial publishers. Your draft should not exceed 1,000 words and should be sent as a Word or Google Doc with the author’s name and title at the top. Agenda blogs allow a maximum of two authors, and sources must be hyperlinked, not referenced as footnotes.

7. **Post-Submission Process:** Major edits and queries will be shared with you before publishing, but minor changes for clarity, flow, and house style will be made without approval. The editorial team carefully selects headlines and images, retaining editorial control, but you are welcome to make suggestions.

8. **Promotion:** Once your blog is published, you'll receive a link to it. You're encouraged to share this link on social media and republish the text in full on your own or your organization’s blog, with a link back to the original article on the Agenda website.

For any questions or further clarifications, you can email the ForumEditorialTeam@weforum.org.